In [1]:
%load_ext autoreload
%autoreload

import deep_nmmo
from deep_nmmo.envs.team_based_env.teams.custom_team import CustomTeam
from deep_nmmo.utils import get_class_from_path, get_module_from_path
from deep_nmmo.envs.team_based_env.env_configs.custom_competition_config import CustomCompetitionConfig
from deep_nmmo.envs.team_based_env.loops.utils import init_env_params, reset_teams, reset_env

import nmmo
from nmmo import config
from nmmo.io import action
from nmmo import scripting, material, Serialized
from nmmo.systems import skill, item
from nmmo.lib import colors
from nmmo import action as Action


import neurips2022nmmo
from neurips2022nmmo.scripted import baselines
from neurips2022nmmo import Team
from neurips2022nmmo import CompetitionConfig, scripted, RollOut, TeamBasedEnv
from neurips2022nmmo.scripted import attack, move

import ray
ray.shutdown()
ray.init()

from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

# from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.policy import PolicySpec

from ray.rllib.models.modelv2 import restore_original_dimensions
from collections.abc import Mapping

from ray.rllib.utils.annotations import override
from ray.rllib.utils.typing import (
    # AgentID,
    # EnvCreator,
    # EnvID,
    # EnvType,
    MultiAgentDict,
    # MultiEnvDict,
)

import gym

import numpy as np

import copy
from collections import defaultdict

from IPython.display import display

from typing import Dict, Any, Type, List

2022-09-29 20:06:56,635	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
2022-09-29 20:07:03,400	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 


### TODO

- Create RLLibScriptedBaseline class to wrap around neurips2022nmmo.scripted.baselines agents so that agents are compatible with RLlib environments. Should inherit from RLlib Policy base class
     - May need to have wrapper class for Team as well. Need to ensure all this still works when making submission

In [2]:
%autoreload

from deep_nmmo.envs.rllib_multi_agent_team_based_env.env import RLlibMultiAgentTeamBasedEnv
from deep_nmmo.envs.rllib_multi_agent_team_based_env.agents.rllib_random_policy import RLlibRandomPolicy

    
    
# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
# teams_config = {
#     'Combat':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
#         },
#     'Mixture':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
#         },
# }
# rllib_paths_to_scripted_agents_cls = {
#                                     '0': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '1': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '2': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '3': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '4': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '5': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     '6': 'neurips2022nmmo.scripted.baselines.Mage',
#                                     }

num_players_per_team = 8
# will overwrite agents kwargs with observation and action space and player index when initialising teams in RLlib env
custom_team_kwargs = {'agents_cls': [RLlibRandomPolicy for _ in range(num_players_per_team)], 
                      'agents_kwargs': [None for _ in range(num_players_per_team)],
                     }

teams_config = {
    # 'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    
    # 'Combat-1': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 1}},
    # 'Combat-2': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 2}},
    # 'Mixture-1': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {'team_id': 3}},
    
    'R-1': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-2': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-3': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    
    
}
# teams_copies = [1, 2]        

/home/zciccwf/personal/projects/neurips2022-nmmo-starter-kit/deep_nmmo/deep_nmmo/envs/rllib_multi_agent_team_based_env/utils.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import defaultdict, Mapping


In [3]:
%autoreload

env_config = get_class_from_path(path_to_env_config_cls)()
ma_env = RLlibMultiAgentTeamBasedEnv(env_config=env_config,
                                     # path_to_env_cls=path_to_env_cls,
                                     # path_to_env_config_cls=path_to_env_config_cls,
                                     teams_config=teams_config)
observations = ma_env.reset()
teams, agent_id_to_agent = ma_env.teams, ma_env.agent_id_to_agent
print(f'')
# print(f'RESET\nobs:\n{obs}\nteams:\n{teams}')
print(f'Reset teams and obs')
step_counter = 1
while observations:
    print(f'\nStep {step_counter}')
    print(f'Step observation keys: {observations.keys()}')
    # agent_to_actions = {agent_id: agent(observations[agent_id]) for agent_id, agent in agent_id_to_agent.items()}
    
    agent_to_actions = {}
    for agent_id in observations.keys():
        # print(f'\nagent_id: {agent_id}')
        # print(f'agent: {agent}')
        agent = agent_id_to_agent[agent_id]
        # actions = agent(observations[agent_id])
        # actions = agent.compute_single_action(observations[agent_id])
        actions, state_outs, info = agent.compute_actions(observations[agent_id])
        print(f'agent ob: {agent.ob}')
        # print(f'actions: {actions}')
        agent_to_actions[agent_id] = actions
    
    observations, rewards, dones, infos = ma_env.step(agent_to_actions)
    print(f'Completed step')
    break # DEBUG
    step_counter += 1
    
print(f'\nEPISODE COMPLETED!!!')

  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:03, 12.25it/s]

Generating 40 maps


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 39/40 [00:02<00:00, 13.05it/s]

KeyboardInterrupt



In [12]:
%autoreload

from deep_nmmo.envs.rllib_multi_agent_team_based_env.env import RLlibMultiAgentTeamBasedEnv
from deep_nmmo.envs.rllib_multi_agent_team_based_env.agents.rllib_random_policy import RLlibRandomPolicy


path_to_rllib_trainer_cls = 'ray.rllib.agents.ppo.PPOTrainer'

path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'

num_players_per_team = 8
# will overwrite agents kwargs with observation and action space and player index when initialising teams in RLlib env
custom_team_kwargs = {'agents_cls': [RLlibRandomPolicy for _ in range(num_players_per_team)], 
                      'agents_kwargs': [None for _ in range(num_players_per_team)],
                     }

teams_config = {
    # 'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    
    # 'Combat-1': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 1}},
    # 'Combat-2': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 2}},
    # 'Mixture-1': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {'team_id': 3}},
    
    'R-1': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-2': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-3': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    
    
}

env_config = get_class_from_path(path_to_env_config_cls)()

policies = {}
policy_id = 1
num_teams, num_players_per_team = 3, 8
for _ in range(num_teams):
    player_idx = 0
    for _ in range(num_players_per_team):
        policies[str(policy_id)] = PolicySpec(
                                    policy_class=RLlibRandomPolicy, # infer from env
                                    # observation_space=dummy_env.observation_space(player_id),
                                    # action_space=dummy_env.action_space(player_id),
                                    config={'env_config': env_config, 'idx': player_idx},
                                )
        policy_id += 1
        player_idx += 1

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    '''Maps agent ID (int) to corresponding policy ID (str) of policy which agent should use.'''
    print(f'>>> Mapping agent_id {agent_id} to {str(agent_id)}')
    return str(agent_id)

multiagent_config = {
    'policies': policies,
    'policy_mapping_fn': policy_mapping_fn,
    'policies_to_train': [],
}

env_config = {
    # 'path_to_env_cls': path_to_env_cls,
    # 'path_to_env_config_cls': path_to_env_config_cls,
    'env_config': env_config,
    'teams_config': teams_config,
}


rllib_config = {
    'env': 'ma_env',
    
    'env_config': env_config,
    
    'disable_env_checking': True,
    # 'disable_env_checking': False,
    
    'framework': 'torch',
    
    'multiagent': multiagent_config,
    
    'num_workers': 0,
    
}

In [13]:
%autoreload

# register env with ray
register_env('ma_env', lambda env_config: RLlibMultiAgentTeamBasedEnv(**env_config))

In [18]:
%autoreload

# merge rllibg trainer's default config with specified config
path_to_agent = '.'.join(path_to_rllib_trainer_cls.split('.')[:-1])
_rllib_config = get_module_from_path(path_to_agent).DEFAULT_CONFIG.copy()
_rllib_config.update(rllib_config)
# print(_rllib_config)

# init rllib trainer
trainer = get_class_from_path(path_to_rllib_trainer_cls)(config=_rllib_config)
print(f'Initialised trainer')

2022-09-29 20:32:21,075	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 13.33it/s]

Generating 40 maps


  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 14.84it/s]

Generating 40 maps


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 14.14it/s]
2022-09-29 20:32:28,765	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Initialised trainer


In [19]:
%autoreload

# perform one training epoch
results = trainer.train()
print(f'Completed RLlib epoch!!!')


~~~ RESET ~~~
RLlibMultiAgentTeamBasedEnv obs keys: 24 dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
>>> Mapping agent_id 1 to 1
>>> Mapping agent_id 2 to 2
>>> Mapping agent_id 3 to 3
>>> Mapping agent_id 4 to 4
>>> Mapping agent_id 5 to 5
>>> Mapping agent_id 6 to 6
>>> Mapping agent_id 7 to 7
>>> Mapping agent_id 8 to 8
>>> Mapping agent_id 9 to 9
>>> Mapping agent_id 10 to 10
>>> Mapping agent_id 11 to 11
>>> Mapping agent_id 12 to 12
>>> Mapping agent_id 13 to 13
>>> Mapping agent_id 14 to 14
>>> Mapping agent_id 15 to 15
>>> Mapping agent_id 16 to 16
>>> Mapping agent_id 17 to 17
>>> Mapping agent_id 18 to 18
>>> Mapping agent_id 19 to 19
>>> Mapping agent_id 20 to 20
>>> Mapping agent_id 21 to 21
>>> Mapping agent_id 22 to 22
>>> Mapping agent_id 23 to 23
>>> Mapping agent_id 24 to 24
rllib_actions obs of agent: (1, 10939) [[  1.   1.   0. ...  40. 277.  15.]]
rllib_actions returned by agent: [defaultdict(<class 'dict'>, {<cl

/scratch/zciccwf/py36/envs/nmmo/lib/python3.9/site-packages/ray/rllib/evaluation/collectors/agent_collector.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(v)


ValueError: all input arrays must have the same shape

In [ ]:


class RLlibScriptedHybridAgentTeam(Team):
    def __init__(self,
                 team_id,
                 env_config,
                 paths_to_scripted_agents_cls,
                 **kwargs):
        super().__init__(team_id, env_config)
        self.id = team_id
        print(f'team_id: {self.id}')
        
        self.scripted_agents = [get_class_from_path(path_to_scripted_agent_cls)(config=env_config, idx=int(idx)) for idx, path_to_scripted_agent_cls in paths_to_scripted_agents_cls.items()]
        self.scripted_agent_ids = set([int(idx) for idx in paths_to_scripted_agents_cls.keys()])
        self.scripted_agent_idxs = set([idx for idx in range(len(self.scripted_agents))])
        
        self.rllib_agents = [None]
        self.rllib_agent_ids = set([sorted(self.scripted_agent_ids)[-1]+1])
        self.rllib_agent_idxs = set([len(self.scripted_agents)+idx for idx in range(1, len(self.rllib_agents)+1)])
        
        print(f'scripted_agents: {self.scripted_agents}')
        print(f'scripted_agent_ids: {self.scripted_agent_ids}')
        print(f'scripted_agent_idxs: {self.scripted_agent_idxs}')
        
        print(f'rllib_agents: {self.rllib_agents}')
        print(f'rllib_agent_ids: {self.rllib_agent_ids}')
        print(f'rllib_agent_idxs: {self.rllib_agent_idxs}')
        
        # init multi-agent env for team to interact with
        agent_ids_to_agents = {}
        for agent_id, agent in zip(self.scripted_agent_ids, self.scripted_agents):
            agent_ids_to_agents[agent_id] = agent
        for agent_id, agent in zip(self.rllib_agent_ids, self.rllib_agents):
            agent_ids_to_agents[agent_id] = agent
        self.team_env = RLlibMultiAgentTeamEnv(
                                            env_config=env_config,
                                            agent_ids_to_agents=agent_ids_to_agents,
                                            team_id=self.id,
                                        )
        print(f'team_env: {self.team_env}')
        
    def reset(self, *args, **kwargs):
        pass
    
    def get_rllib_agent_actions(self, observations):
        return {i: self.agents[i](obs) for i, obs in observations.items() if i in self.rllib_agent_idxs}
    
    def get_scripted_agent_actions(self, observations):
        return {i: self.agents[i](obs) for i, obs in observations.items() if i in self.scripted_agent_idxs}
    
    def post_process_actions(self, actions):
        for i in actions:
            for atn, args in actions[i].items():
                for arg, val in args.items():
                    if arg.argType == nmmo.action.Fixed:
                        actions[i][atn][arg] = arg.edges.index(val)
                    elif arg == nmmo.action.Target:
                        actions[i][atn][arg] = self.get_target_index(
                            val, self.agents[i].ob.agents)
                    elif atn in (nmmo.action.Sell,
                                 nmmo.action.Use) and arg == nmmo.action.Item:
                        actions[i][atn][arg] = self.get_item_index(
                            val, self.agents[i].ob.items)
                    elif atn == nmmo.action.Buy and arg == nmmo.action.Item:
                        actions[i][atn][arg] = self.get_item_index(
                            val, self.agents[i].ob.market)
        return actions
    
    def act(self, observations, rewards=None):
        '''
        During training, pass rewards from last step to update RLlib policy.
        
        During inference (e.g. when make submission), no rewards need to be
        passed to act().
        '''
        if "stat" in observations:
            stat = observations.pop("stat")
            
        if rewards is None:
            # agent is training, register rewards
            # if an action was chosen at the last step, assign team reward for taking that action
            # TODO
            pass
        else:
            # not training, no need to consider rewards
            pass
        
        # TODO
        # get team actions for this step
        actions = {}
        actions.update(self.get_scripted_agent_actions(observations))
        actions.update(self.get_rllib_agent_actions(observations))
        
        # return team actions to TeamBasedEnv
        return self.post_process_actions(actions)

    @staticmethod
    def get_item_index(instance: int, items: np.ndarray) -> int:
        for i, itm in enumerate(items):
            id_ = nmmo.scripting.Observation.attribute(itm,
                                                       nmmo.Serialized.Item.ID)
            if id_ == instance:
                return i
        raise ValueError(f"Instance {instance} not found")

    @staticmethod
    def get_target_index(target: int, agents: np.ndarray) -> int:
        targets = [
            x for x in [
                nmmo.scripting.Observation.attribute(
                    agent, nmmo.Serialized.Entity.ID) for agent in agents
            ] if x
        ]
        return targets.index(target)
        
        
        
        
        
# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
# teams_config = {
#     'Combat':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
#         },
#     'Mixture':
#         {
#             'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
#         },
# }
rllib_paths_to_scripted_agents_cls = {
                                    '0': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '1': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '2': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '3': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '4': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '5': 'neurips2022nmmo.scripted.baselines.Mage',
                                    '6': 'neurips2022nmmo.scripted.baselines.Mage',
                                    }

teams_config = {
    'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    'Combat': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {}},
    'Mixture': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {}},
}
teams_copies = [1, 2]        
        
        
        
        
        
# init env params
env_config = get_class_from_path(path_to_env_config_cls)()

teams = []
for team_id, params in teams_config.items():
    team_cls, team_kwargs = params['cls'], params['kwargs']
    team_kwargs['env_config'] = env_config
    team_kwargs['team_id'] = team_id
    teams.append(team_cls(**team_kwargs))
print(f'Teams: {teams}')

for i, team in enumerate(teams):
    class Agent(nmmo.Agent):
        name = f'{team.id}'
        policy = f'{team.id}'
    env_config.PLAYERS[i] = Agent

In [ ]:
test = {1: 3, 2: 2, 4: 5}
print(len(test))

In [ ]:
# set up configs

# env config
path_to_env_cls = 'neurips2022nmmo.TeamBasedEnv'
path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'
env_config_kwargs = None
teams_config = {
    'Combat':
        {
            'path_to_team_cls': 'neurips2022nmmo.scripted.CombatTeam'
        },
    'Mixture':
        {
            'path_to_team_cls': 'neurips2022nmmo.scripted.MixtureTeam'
        },
}
teams_copies = [1, 2]

# init env params
env_config, teams_copies, teams = init_env_params(path_to_env_config_cls=path_to_env_config_cls,
                                                  env_config_kwargs=env_config_kwargs,
                                                  teams_copies=teams_copies,
                                                  teams_config=teams_config)



# rllib config
path_to_rllib_trainer_cls = 'ray.rllib.agents.ppo.PPOTrainer'

# ma_cls = make_multi_agent(lambda env_config: get_class_from_path(path_to_env_cls)(env_config))
# print(ma_cls)
# dummy_env = ma_cls(env_config)
# print(dummy_env)
# print(type(dummy_env))
# _ = dummy_env.reset()


dummy_env = RLlibMultiAgentTeamBasedEnv(
                                    path_to_env_cls=path_to_env_cls,
                                    path_to_env_config_cls=path_to_env_config_cls,
                                    teams_copies=teams_copies,
                                    teams_config=teams_config,
                                )
print(dummy_env)

policies = {}
for policy_id in dummy_env._agent_ids:
    policies[str(policy_id)] = PolicySpec(
                                # policy_class=player.__class__, # infer automatically from algorithm
                                # observation_space=dummy_env.observation_space(player_id),
                                # action_space=dummy_env.action_space(player_id),
                                config={},
                            )

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    '''Maps agent ID to corresponding policy ID of policy which agent should use.'''
    return str(agent_id)

multiagent_config = {
    'policies': policies,
    'policy_mapping_fn': policy_mapping_fn,
}


rllib_config = {
    'framework': 'torch',
    
    'multiagent': multiagent_config,
    
}

In [ ]:
# register env with ray
register_env(path_to_env_cls.split('.')[-1], lambda env_config: make_multi_agent(get_class_from_path(path_to_env_cls))(env_config))

In [ ]:
# # update rllib config with observation and action spaces
# dummy_env = get_class_from_path(path_to_env_cls)(env_config)
# agent = 0
# rllib_config['observation_space'] = dummy_env.observation_space(agent)
# rllib_config['action_space'] = dummy_env.action_space(agent)

# init rllib trainer
trainer = get_class_from_path(path_to_rllib_trainer_cls)(config=rllib_config)

In [ ]:
env = TeamBasedEnv(env_config)

In [ ]:
print(env.observation_space(0))
print(type(env.observation_space(0)))

In [ ]:
for key, val in env.observation_space(0).items():
    print(f'\nkey: {key}')
    print(f'val: {val}')

In [ ]:
import copy

edges = [0, 1, 2, 3]
for edge in edges:
    if edge == 1:
        max_edge = edge
print(f'edge: {edge} | max_edge: {max_edge}')

In [ ]:
a = [i for i in range(-1)]
print(a)

In [ ]:
print(0 < len([]))

In [ ]:
import copy
import time

class Job:
    def __init__(self,
                 job_id,
                 original_job=None):
        self.job_id = job_id
        if original_job is None:
            self.original_job = copy.deepcopy(self)
        else:
            self.original_job = original_job
        
job_id = 0
original_job = Job(job_id=job_id, original_job=None)
print(f'original_job job_id: {original_job.job_id} | original_job job_id: {original_job.original_job.job_id}')

print('')
job_id = 1
start_t = time.time()
# _original_job = copy.deepcopy(original_job)
_original_job = copy.copy(original_job)
# _original_job = original_job
_original_job.job_id = job_id
print(f'Updated _original_job in {time.time() - start_t} s')
job_1 = Job(job_id=1, original_job=_original_job)
print(f'original_job job_id: {original_job.job_id} | original_job job_id: {original_job.original_job.job_id}')
print(f'job_1 job_id: {job_1.job_id} | original_job job_id: {job_1.original_job.job_id}')

In [ ]:
print(type(-1))